In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
def prepare_df():

    file_path = '/Users/julioberrocal/Desktop/Classes/Winter 2024/Advanced Programming/Final Project/Muffintown_Final/Query for BABSON Project-2.xlsx'
    sheetname = '2. Line Performance'
    df = pd.read_excel(file_path, sheet_name=sheetname)
    
    # Converting columns to Datetime
    df['Start of Batch Date/Time'] = pd.to_datetime(df['Start of Batch Date/Time'])
    df['End of Batch Date/Time'] = pd.to_datetime(df['End of Batch Date/Time'])

    # Calculating the batch length in hours
    df['Batch Length (Hours)'] = (df['End of Batch Date/Time'] - df['Start of Batch Date/Time']).dt.total_seconds() / 3600

    # Creating Units per Hour Column
    df['Units Per Hour'] = df['Batch Count'] / df['Batch Length (Hours)']

    # Remove rows with missing values in 'Optimal Cases Per Hour' column
    df.dropna(subset=['Optimal Cases Per Hour'], inplace=True)

    compliant_df = df[(df['Units Per Hour'] >= df['Lower Target']) & (df['Units Per Hour'] <= df['Upper Target'])]
    non_compliant_df = df[(df['Units Per Hour'] < df['Lower Target']) | (df['Units Per Hour'] > df['Upper Target'])]

    total_observations = len(df)
    print(total_observations)
    pct_compliant = 100 * round(len(compliant_df)/total_observations,2)
    pct_non_compliant = 100 * round(len(non_compliant_df)/total_observations,2)

    print(f'The number of compliant observations is: {len(compliant_df)} or {pct_compliant} percent.')
    print(f'The number of non-compliant observations is: {len(non_compliant_df)} or {pct_non_compliant} percent.')

    return df

initial_df = prepare_df()

2939
The number of compliant observations is: 429 or 15.0 percent.
The number of non-compliant observations is: 2510 or 85.0 percent.


In [ ]:
def analyze_sku(df, skus):
    analysis_results = []
    
    for sku in skus:
        # Filter by the current SKU
        filtered_df = df[df['Current Job'] == sku]
        
        # Number of records for the SKU
        records = len(filtered_df)
        
        # Percentage of compliant records for the SKU
        compliant_records = len(filtered_df[(filtered_df['Units Per Hour'] >= filtered_df['Lower Target']) & 
                                                (filtered_df['Units Per Hour'] <= filtered_df['Upper Target'])])
        
        compliance_percentage = round((compliant_records / records),2) * 100 if records != 0 else 0
        
        # Average units per hour of the SKU
        avg_units_per_hour = round(filtered_df['Units Per Hour'].mean(),2)
        
        # Standard deviation of units per hour of the SKU
        stdev_units_per_hour = round(filtered_df['Units Per Hour'].std(),2)

        # Plot histogram of units per hour for the SKU
        plt.figure(figsize=(8, 6))
        plt.hist(filtered_df['Units Per Hour'], bins=20, color='skyblue', edgecolor='black')
        plt.title(f'Units Per Hour Distribution for SKU: {sku}')
        plt.xlabel('Units Per Hour')
        plt.ylabel('Frequency')
        plt.grid(True)
        plt.savefig(f'{sku}_histogram.png')

        analysis_results.append({
            'SKU': sku,
            'Number of Records': records,
            'Percentage of Compliant Records': compliance_percentage,
            'Average Units Per Hour': avg_units_per_hour,
            'Standard Deviation of Units Per Hour': stdev_units_per_hour
        })
    
    return analysis_results

skus = ['96605', '24970']
sku_results = analyze_sku(initial_df, skus)
for result in sku_results:
    print(result)


In [23]:
def analyze_group(df, group_names):
    analysis_results = []
    
    for group_name in group_names:
        # Filter by the current Group Name
        filtered_df = df[df['Group Name'] == group_name]
        
        # Number of records for the Group Name
        records = len(filtered_df)
        
        # Percentage of compliant records for the Group Name
        compliant_records = len(filtered_df[(filtered_df['Units Per Hour'] >= filtered_df['Lower Target']) & 
                                                (filtered_df['Units Per Hour'] <= filtered_df['Upper Target'])])
        compliance_percentage = round((compliant_records / records),2) * 100 if records != 0 else 0
        
        analysis_results.append({
            'Group Name': group_name,
            'Number of Records': records,
            'Percentage of Compliant Records': compliance_percentage,
        })
    
    return analysis_results

group_names = ['LAWRENCE AUTOBAKE', 'LAWRENCE - SW PACK']
group_results = analyze_group(initial_df, group_names)
for result in group_results:
    print(result)


{'Group Name': 'LAWRENCE AUTOBAKE', 'Number of Records': 2200, 'Percentage of Compliant Records': 19.0}
{'Group Name': 'LAWRENCE - SW PACK', 'Number of Records': 471, 'Percentage of Compliant Records': 1.0}
